### Libraries

In [1]:
# packages to install

from bs4 import BeautifulSoup
from tqdm import tqdm
import pandas as pd
import requests
import re
from datetime import datetime
import pathlib


In [2]:
url = 'https://ecf.dcd.uscourts.gov/cgi-bin/CourtSched.pl'
page = requests.get(url)



<Response [200]>


In [3]:
soup = BeautifulSoup(page.text, 'html.parser')

In [4]:
table = soup.find("table", {"id":"ts"})
date = soup.find("b").text.split(":")[1]
date = " ".join(date.split())
formatted_date = datetime.strptime(date, '%A, %B %d, %Y')


datetime.datetime(2022, 7, 21, 0, 0)

In [5]:
ls_table_tr = table.find_all("tr")
rows = []
for tr in ls_table_tr:
    row = []
    count = 0
    for child in tr.children:
        if count == 0:
            result = child.text.replace('\n', '')
            if result != "":
                 row.append(result)
        else: 

            try:
                row.append(child.text.replace('\n', ''))
            except:
                continue

    count = count + 1
    if len(row) > 0:
        rows.append(row)
df = pd.DataFrame(rows[1:], columns = rows[0])

In [6]:
df[['Case Number', 'Title']] = df['Case Number and Title'].str.split(':', expand=True)

In [7]:
df["Date"] = formatted_date

In [ ]:
df

In [18]:

file_date = formatted_date.strftime("%Y-%m-%d")
path = pathlib.Path(f"../data/days/{file_date}-court_calendar.csv")
df.to_csv(path, index = False)